In [1]:
import os
import cv2
from pathlib import Path
import random
from tqdm import tqdm


In [2]:
BASE_DIR = Path.cwd()
DATASET_PATH = BASE_DIR / 'dataset'
CATEGORIES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
TARGET_COUNT = 500


In [3]:
def getImages(categoryPath):
    return list(categoryPath.glob('*.jpg'))


def printCategoryCounts():
    print("\nImage counts per category:")
    print("-" * 30)
    for categoryName in CATEGORIES:
        categoryPath = DATASET_PATH / categoryName
        imageCount = len(getImages(categoryPath))
        print(f"{categoryName:15s}: {imageCount} images")


def validateAndCleanDataset():
    if not DATASET_PATH.exists():
        raise FileNotFoundError(f"Dataset not found at: {DATASET_PATH}")
    print("Validating dataset for corrupted images...")
    categoryDirectories = [directory for directory in DATASET_PATH.iterdir() if directory.is_dir()]
    print(f"Found {len(categoryDirectories)} categories: {[category.name for category in categoryDirectories]}\n")
    totalRemovedImages = 0
    for categoryDirectory in sorted(categoryDirectories):
        imageFiles = [file for file in categoryDirectory.iterdir() if file.is_file()]
        removedCount = 0
        for imagePath in imageFiles:
            if cv2.imread(str(imagePath)) is None:
                print(f"Removing: {imagePath.name} as it is corrupted.")
                try:
                    os.remove(imagePath)
                    removedCount += 1
                except Exception as error:
                    print(f"Warning: Could not remove: {error}")
        numberOfValidImages = len(imageFiles) - removedCount
        print(f"{categoryDirectory.name}: {numberOfValidImages}/{len(imageFiles)} valid images")
        totalRemovedImages += removedCount
    print(f"\nValidation complete. Removed {totalRemovedImages} corrupted file(s).")
    printCategoryCounts()


def augmentImage(image):
    height, width = image.shape[:2]
    center = (width / 2, height / 2)
    augmentations = [
        cv2.flip(image, 1),  # Horizontal flip
        cv2.flip(image, 0),  # Vertical flip
        cv2.warpAffine(image, cv2.getRotationMatrix2D(center, 90, 1.0), (width, height)), # 90 degrees
        cv2.warpAffine(image, cv2.getRotationMatrix2D(center, 180, 1.0), (width, height)), # 180 degrees
        cv2.warpAffine(image, cv2.getRotationMatrix2D(center, 270, 1.0), (width, height)), # 270 degrees
        cv2.convertScaleAbs(image, alpha=1.3, beta=30),   # Brightness +30
        cv2.convertScaleAbs(image, alpha=0.7, beta=-30),  # Brightness -30
        cv2.GaussianBlur(image, (5, 5), 0),               # Gaussian blur 5x5
    ]
    # Zoom crop
    scale = 1.2
    newHeight, newWidth = int(height * scale), int(width * scale)
    resized = cv2.resize(image, (newWidth, newHeight))
    startHeight, startWidth = (newHeight - height) // 2, (newWidth - width) // 2
    augmentations.append(resized[startHeight:startHeight + height, startWidth:startWidth + width])
    return augmentations


def augmentDataset():
    print(f"\nAugmenting images to reach {TARGET_COUNT} per category...")
    for categoryName in CATEGORIES:
        categoryPath = DATASET_PATH / categoryName
        imagePaths = getImages(categoryPath)
        currentImageCount = len(imagePaths)
        imagesNeeded = TARGET_COUNT - currentImageCount
        print(f"\n{categoryName}: {currentImageCount} images", end="")
        if imagesNeeded <= 0:
            print(" - Already sufficient")
            continue
        print(f" (Need {imagesNeeded} more)")
        # Load original images using tqdm for progress
        originalImages = [(cv2.imread(str(imageFile)), imageFile.stem) for imageFile in tqdm(imagePaths, desc="Loading")]
        originalImages = [(image, name) for image, name in originalImages]
        # Generate augmented images
        generatedCount = 0
        while generatedCount < imagesNeeded:
            image, imageName = random.choice(originalImages)
            for augmentationIndex, augmentedImage in enumerate(augmentImage(image)):
                if generatedCount >= imagesNeeded:
                    break
                savePath = categoryPath / f"{imageName}_augmented_{generatedCount}_{augmentationIndex}.jpg"
                cv2.imwrite(str(savePath), augmentedImage)
                generatedCount += 1
        print(f"Generated {generatedCount} augmented images")
    print("\n" + "=" * 50)
    print("Augmentation complete.")
    printCategoryCounts()



In [4]:
validateAndCleanDataset()


Validating dataset for corrupted images...
Found 6 categories: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

Removing: 2ec9d19b-8027-4c77-a13f-5eee033b9868.jpg as it is corrupted.
Removing: 31381a44-38d6-4a44-9384-7690727801bc.jpg as it is corrupted.
Removing: 345bdb67-4190-4235-a16f-b60c1556a28d.jpg as it is corrupted.
Removing: 38b3e4da-738c-4694-a946-55101b25ad53.jpg as it is corrupted.
Removing: 4840d678-7af4-4a2d-bda1-338c2f2a59c5.jpg as it is corrupted.
Removing: 509251d8-4e3a-4f1e-aabc-4d034b0f2455.jpg as it is corrupted.
Removing: 5b7da318-c2ab-4c29-8ace-19895a890840.jpg as it is corrupted.
Removing: 8617221e-dc90-48fe-a116-46350b5f814e.jpg as it is corrupted.
Removing: 88ce5fbf-e9c7-40ad-87a6-deffe95d8ee8.jpg as it is corrupted.
Removing: bff223bf-1a84-4d38-a486-c3f4c9bfef5e.jpg as it is corrupted.
Removing: ce8a4c3d-2a08-4e78-9e5a-16b69719e505.jpg as it is corrupted.
Removing: d5856b01-c157-4e34-b921-80f29252976a.jpg as it is corrupted.
cardboard: 247/259 vali

In [5]:
augmentDataset()



Augmenting images to reach 500 per category...

cardboard: 247 images (Need 253 more)


Loading: 100%|██████████| 247/247 [00:00<00:00, 465.75it/s]


Generated 253 augmented images

glass: 385 images (Need 115 more)


Loading: 100%|██████████| 385/385 [00:00<00:00, 519.76it/s]


Generated 115 augmented images

metal: 315 images (Need 185 more)


...

Generated 51 augmented images

plastic: 363 images (Need 137 more)


Loading: 100%|██████████| 363/363 [00:00<00:00, 611.24it/s]


Generated 137 augmented images

trash: 106 images (Need 394 more)


Loading: 100%|██████████| 106/106 [00:00<00:00, 660.24it/s]


Generated 394 augmented images

Augmentation complete!

Image counts per category:
------------------------------
cardboard      : 500 images
glass          : 500 images
metal          : 500 images
paper          : 500 images
plastic        : 500 images
trash          : 500 images
